## This pattern introduces quality control into workflows by having dedicated evaluation steps that assess intermediate results. Based on the evaluation, the workflow can either proceed, retry with adjusted parameters, or take corrective action. This creates more robust workflows capable of self-improvement and error recovery.


In [5]:
import { createCompatibleOpenAI } from '../../init.ts'
import { generateText, generateObject } from 'ai'
import { z } from 'zod'

const openai = createCompatibleOpenAI()

const evaluationSchema = z.object({
  qualityScore: z.number().min(1).max(10),
  preservesTone: z.boolean(),
  preservesNuance: z.boolean(),
  culturallyAccurate: z.boolean(),
  specificIssues: z.array(z.string()),
  improvementSuggestions: z.array(z.string())
})

async function translateWithFeedback(text: string, targetLanguage: string) {
  let currentTranslation = ''
  let iterations = 0
  const MAX_ITERATIONS = 3

  // Initial translation
  const { text: translation } = await generateText({
    model: openai('gpt-4o-mini'), // use small model for first attempt
    system: 'You are an expert literary translator.',
    prompt: `Translate the following text to ${targetLanguage}, preserving tone and cultural nuances: ${text}`
  })

  currentTranslation = translation
  iterations++

  // Evaluation-optimization loop
  while (iterations < MAX_ITERATIONS) {
    // Evaluate current translation
    const { object: evaluation } = await generateObject<z.infer<typeof evaluationSchema>>({
      model: openai('gpt-4o'), // use larger model for evaluation
      system: 'You are an expert literary translator.',
      schema: evaluationSchema,
      prompt: `Evaluate this translation:

      Original: ${text}
      Translation: ${currentTranslation}

      Consider:
      1. Overall quality
      2. Preservation of tone
      3. Preservation of nuance
      4. Cultural accuracy`
    })

    // Check if quality meets threshold
    if (
      evaluation.qualityScore >= 8 &&
      evaluation.preservesTone &&
      evaluation.preservesNuance &&
      evaluation.culturallyAccurate
    ) {
      break
    }

    // Generate improved translation based on feedback
    const { text: improvedTranslation } = await generateText({
      model: openai('gpt-4o'), // use a larger model
      system: 'You are an expert literary translator.',
      prompt: `Improve this translation based on the following feedback:
					${evaluation.specificIssues.join('\n')}
					${evaluation.improvementSuggestions.join('\n')}
		
					Original: ${text}
					Current Translation: ${currentTranslation}`
    })

		currentTranslation = improvedTranslation
		iterations++
  }

	return {
		finalTranslation: currentTranslation,
		iterationsRequired: iterations
	}
}


const text = '欲买桂花同载酒，终不似，少年游。'
const targetLanguage = 'English'

const result = await translateWithFeedback(text, targetLanguage)

console.log(result)



{
  finalTranslation: "The desire to buy osmanthus and share wine, ultimately can't compare to the carefree days of youth.",
  iterationsRequired: 1
}
